In [ ]:
!pip install datasets
!pip install tqdm
!pip install pandas
!pip install rouge_score
!pip install bert_score
!pip install py7zr

In [3]:
from datasets import load_dataset, load_metric
from tqdm import tqdm
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
'''
file_path points to a fie in this folder with summaries separated by '\n' (1 summary/row).
The rouge metric is evaluated over the test set of samsum dataset.
Change the paths according to your drive folder structure
'''

file_path = '/content/drive/MyDrive/2_1_Deep_NLP/SICK_Summarization/eval_trials/tmp_result_EMOJI_M1.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    candidates = file.read().split('\n')

candidates = candidates[:-1] # the last one is an empy new line, drop it.

samsum = load_dataset('samsum')
samsum_test_summary = samsum['test']['summary']

FileNotFoundError: ignored

In [ ]:
metric = load_metric('/content/drive/MyDrive/2_1_Deep_NLP/SICK_Summarization/utils/rouge.py')
bertscore_metric = load_metric("bertscore", lang='en', model_type='bert-base-uncased')

results_v2=[]
for i in tqdm(range(len(candidates))):

    r = metric.compute(predictions=[candidates[i]], references=[samsum_test_summary[i]])
    r = {key: value.mid.fmeasure * 100 for key, value in r.items()}

    bertscore_metric.add_batch(predictions=[candidates[i]], references=[samsum_test_summary[i]])
    bertscore_result = bertscore_metric.compute(lang='en', model_type='bert-base-uncased')
    bertscore_result = sum(bertscore_result['f1']) / len(bertscore_result['f1'])
    
    r['bert_score'] = bertscore_result

    results_v2.append(r)

h:\Il mio Drive\2_1_Deep_NLP\trials\rouge_clean_emoji_M1.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  from datasets import load_dataset
100%|██████████| 819/819 [01:55<00:00,  7.08it/s]


In [ ]:
res_v2 = pd.DataFrame([samsum_test_summary, candidates, results_v2]).T
res_v2.columns = ['samsum_test_summary', 'candidates', 'results']
res_v2 = res_v2['results'].apply(pd.Series).merge(res_v2, left_index=True, right_index=True).drop('results', axis=1)

res_v2.to_excel('./res_v2_rouge_scores_EMOJI_M1.xlsx', encoding='utf-8')